In [40]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [41]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import pandas as pd

In [42]:
df_updated=pd.read_csv("text_with_vectors_3.csv")
X=df_updated['vector']
y=df_updated['Score']

In [43]:
from sklearn.model_selection import train_test_split

# Split into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


In [44]:
import numpy as np

X_train = np.array([np.fromstring(vec.strip('[]'), sep=' ') for vec in X_train])


In [45]:
import numpy as np

X_test= np.array([np.fromstring(vec.strip('[]'), sep=' ') for vec in X_test])


In [14]:
# X_train = X_train.to_numpy()
X_test=X_test.to_numpy()

In [15]:
X_test = X_test.reshape(-1, 1)  # Reshape if it's missing a second dimension


In [48]:
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(3, activation='softmax')  # Output layer for 3 classes
])

# Compile the model with a reduced learning rate
model.compile(optimizer=Adam(learning_rate=0.0005),  
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights and fewer epochs
history = model.fit(X_train, y_train, 
                    epochs=50,  # Reduced from 100 to 30
                    batch_size=64, 
                    validation_data=(X_test, y_test), 
                    callbacks=[early_stopping],
                    verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

Epoch 1/50
625/625 [==============================] - 5s 8ms/step - loss: 1.3331 - accuracy: 0.5322 - val_loss: 1.0252 - val_accuracy: 0.6350
Epoch 2/50
625/625 [==============================] - 4s 7ms/step - loss: 1.0477 - accuracy: 0.6204 - val_loss: 0.9650 - val_accuracy: 0.6540
Epoch 3/50
625/625 [==============================] - 4s 7ms/step - loss: 0.9748 - accuracy: 0.6386 - val_loss: 0.9266 - val_accuracy: 0.6557
Epoch 4/50
625/625 [==============================] - 4s 7ms/step - loss: 0.9293 - accuracy: 0.6475 - val_loss: 0.9184 - val_accuracy: 0.6392
Epoch 5/50
625/625 [==============================] - 4s 7ms/step - loss: 0.8981 - accuracy: 0.6530 - val_loss: 0.8943 - val_accuracy: 0.6435
Epoch 6/50
625/625 [==============================] - 4s 7ms/step - loss: 0.8690 - accuracy: 0.6551 - val_loss: 0.8565 - val_accuracy: 0.6582
Epoch 7/50
625/625 [==============================] - 4s 7ms/step - loss: 0.8541 - accuracy: 0.6554 - val_loss: 0.8456 - val_accuracy: 0.6574
Epoch 

In [ ]:
model.save("modelv0.0.h5")